# Assignment 2


You must submit your notebook by running `python3 -m autograder.run.submit Assignment2.ipynb` from your local repository.

To write legible answers you will need to be familiar with both [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet) and [Latex](https://www.latex-tutorial.com/tutorials/amsmath/)

Before you turn this problem in, make sure everything runs as expected. To do so, restart the kernel and run all cells (in the menubar, select Runtime→→Restart and run all).

#### Show your work!
Whenever you are asked to find the solution to a problem, be sure to also **show how you arrived** at your answer.

Make sure you fill in any place that says "YOUR CODE HERE" or "YOUR ANSWERS HERE", as well as your name below:

## Some Helper Functions

Implement an evaluation function that takes in some board position and player colour and returns a score. 

In [33]:
import chess
import random
from math import inf
from IPython.display import display, clear_output

import asyncio
import sys

# This MUST be run before any other asyncio code
if sys.platform == 'win32':
    asyncio.set_event_loop_policy(asyncio.WindowsProactorEventLoopPolicy())

## Q1

Implement the minimax algorithm that chooses the best move for a given board position, arbitrary evaluation function, player colour, and depth.

Make sure to write your implementation to be able to use the provided `evaluation()` function when calculating the value of board states. Later on, you'll be writing your own, but for now, your minimax will be tested using a provided evaluation function on the autograder.

Note about evaluation functions: call the evaluation function in the following manner: `evaluation(board: chess.board, player: bool)`. Think about which player should be passed into the evaluation function; should it be the player who is currently acting (according to the local max/min node) or should it be in accord with the player at the root of the tree?

If you time out during some of the test cases for this problem, consider adding alpha-beta pruning to your implementation.

In [43]:
import math
import chess.polyglot

def get_minimax_move(b: chess.Board, evaluation, player: bool, ply: int):
    """
    This function chooses the best move for the given board position, evaluation function, player, and ply.
    
    Parameters:
    - board: the chess board
    - evaluation: a function that returns a score given a chess board and a player
    - player: the colour of the active player (True -> white, False -> black)
    - ply: the number of move pairs (1 ply = both max AND min) that the algorithmn should look ahead.
    
    Returns:
    A single chess.Move type object.
    
    best_move = None
    best_value = -math.inf
    alpha = -math.inf
    beta = math.inf

    # The depth in half-moves is ply * 2
    depth = ply * 2

    for move in b.legal_moves:
        b.push(move)
        # Call the helper function (minimizing layer comes next)
        board_value = minimax(b, depth - 1, alpha, beta, False, evaluation, player)
        b.pop()

        if board_value > best_value:
            best_value = board_value
            best_move = move
        
        # Update alpha for the root (maximizing)
        alpha = max(alpha, best_value)

    return best_move

def minimax(board, depth, alpha, beta, is_maximizing, evaluation, root_player):
    # Base case: leaf node or game over
    if depth == 0 or board.is_game_over():
        return evaluation(board, root_player)

    if is_maximizing:
        max_eval = -math.inf
        for move in board.legal_moves:
            board.push(move)
            eval = minimax(board, depth - 1, alpha, beta, False, evaluation, root_player)
            board.pop()
            max_eval = max(max_eval, eval)
            alpha = max(alpha, eval)
            if beta <= alpha:
                break  # Beta cutoff
        return max_eval
    else:
        min_eval = math.inf
        for move in board.legal_moves:
            board.push(move)
            eval = minimax(board, depth - 1, alpha, beta, True, evaluation, root_player)
            board.pop()
            min_eval = min(min_eval, eval)
            beta = min(beta, eval)
            if beta <= alpha:
                break  # Alpha cutoff
        return min_eval
        
    best_move = None
    best_value = -math.inf
    alpha = -math.inf
    beta = math.inf
    depth = ply * 2

    # Prioritize moves: Captures first to trigger earlier cutoffs
    ordered_moves = sorted(b.legal_moves, key=lambda m: b.is_capture(m), reverse=True)

    for move in ordered_moves:
        b.push(move)
        board_value = minimax(b, depth - 1, alpha, beta, False, evaluation, player)
        b.pop()

        if board_value > best_value:
            best_value = board_value
            best_move = move
        
        alpha = max(alpha, best_value)

    return best_move

def minimax(board, depth, alpha, beta, is_maximizing, evaluation, root_player):
    if depth == 0 or board.is_game_over():
        return evaluation(board, root_player)

    # Move ordering within the recursion
    moves = sorted(board.legal_moves, key=lambda m: board.is_capture(m), reverse=True)

    if is_maximizing:
        max_eval = -math.inf
        for move in moves:
            board.push(move)
            eval = minimax(board, depth - 1, alpha, beta, False, evaluation, root_player)
            board.pop()
            max_eval = max(max_eval, eval)
            alpha = max(alpha, eval)
            if beta <= alpha:
                break
        return max_eval
    else:
        min_eval = math.inf
        for move in moves:
            board.push(move)
            eval = minimax(board, depth - 1, alpha, beta, True, evaluation, root_player)
            board.pop()
            min_eval = min(min_eval, eval)
            beta = min(beta, eval)
            if beta <= alpha:
                break
        return min_eval
    """ 
    best_move = None
    best_value = -math.inf
    alpha = -math.inf
    beta = math.inf
    depth = ply * 2

    # Better move ordering: MVV-LVA for captures
    ordered_moves = _order_moves(b, list(b.legal_moves))

    for move in ordered_moves:
        b.push(move)
        board_value = minimax(b, depth - 1, alpha, beta, False, evaluation, player)
        b.pop()

        if board_value > best_value:
            best_value = board_value
            best_move = move
        
        alpha = max(alpha, best_value)
        if beta <= alpha:  # Can prune at root too
            break

    return best_move

# Piece values for move ordering
_PIECE_VALUES = {
    chess.PAWN: 100,
    chess.KNIGHT: 320,
    chess.BISHOP: 330,
    chess.ROOK: 500,
    chess.QUEEN: 900,
    chess.KING: 20000
}

def _order_moves(board, moves):
    """Order moves for better alpha-beta pruning"""
    def move_score(move):
        score = 0
        
        # Promotions are valuable
        if move.promotion:
            score += 900
        
        # MVV-LVA for captures
        if board.is_capture(move):
            victim = board.piece_at(move.to_square)
            attacker = board.piece_at(move.from_square)
            if victim and attacker:
                # Capture value = victim_value * 10 - attacker_value
                score += _PIECE_VALUES[victim.piece_type] * 10 - _PIECE_VALUES[attacker.piece_type]
            elif victim:
                score += _PIECE_VALUES[victim.piece_type] * 10
        
        return score
    
    return sorted(moves, key=move_score, reverse=True)

# Transposition table: stores {zobrist_hash: (depth, value, flag, best_move)}
_transposition_table = {}
_TT_EXACT = 0
_TT_ALPHA = 1  # Upper bound
_TT_BETA = 2   # Lower bound

def minimax(board, depth, alpha, beta, is_maximizing, evaluation, root_player):
    alpha_orig = alpha
    
    # Transposition table lookup
    board_hash = chess.polyglot.zobrist_hash(board)
    if board_hash in _transposition_table:
        tt_depth, tt_value, tt_flag, tt_move = _transposition_table[board_hash]
        if tt_depth >= depth:
            if tt_flag == _TT_EXACT:
                return tt_value
            elif tt_flag == _TT_ALPHA:  # Upper bound
                beta = min(beta, tt_value)
            elif tt_flag == _TT_BETA:   # Lower bound
                alpha = max(alpha, tt_value)
            
            if alpha >= beta:
                return tt_value
    
    # Terminal node
    if depth == 0 or board.is_game_over():
        value = evaluation(board, root_player)
        _transposition_table[board_hash] = (depth, value, _TT_EXACT, None)
        return value

    # Get and order moves
    moves_list = list(board.legal_moves)
    
    # Put TT move first if available
    tt_move = _transposition_table.get(board_hash, (0, 0, 0, None))[3]
    if tt_move and tt_move in moves_list:
        moves_list.remove(tt_move)
        moves_list.insert(0, tt_move)
    
    ordered_moves = _order_moves(board, moves_list)
    
    best_move = None
    
    if is_maximizing:
        max_eval = -math.inf
        for move in ordered_moves:
            board.push(move)
            eval_score = minimax(board, depth - 1, alpha, beta, False, evaluation, root_player)
            board.pop()
            
            if eval_score > max_eval:
                max_eval = eval_score
                best_move = move
            
            alpha = max(alpha, eval_score)
            if beta <= alpha:
                break  # Beta cutoff
        
        value = max_eval
    else:
        min_eval = math.inf
        for move in ordered_moves:
            board.push(move)
            eval_score = minimax(board, depth - 1, alpha, beta, True, evaluation, root_player)
            board.pop()
            
            if eval_score < min_eval:
                min_eval = eval_score
                best_move = move
            
            beta = min(beta, eval_score)
            if beta <= alpha:
                break  # Alpha cutoff
        
        value = min_eval
    
    # Store in transposition table
    if value <= alpha_orig:
        tt_flag = _TT_ALPHA  # Upper bound
    elif value >= beta:
        tt_flag = _TT_BETA   # Lower bound
    else:
        tt_flag = _TT_EXACT
    
    _transposition_table[board_hash] = (depth, value, tt_flag, best_move)
    
    return value

## Q2

Implement the expectimax algorithm that chooses the best move for a given board position, arbitrary evaluation function, player colour, and depth.

Make sure to write your implementation to be able to use the provided `evaluation()` function when calculating the value of board states. Later on, you'll be writing your own, but for now, your expectimax will be tested using a provided function on the autograder.

You should handle the evaluation function here in the same way as you did in your minimax implementation: `evaluation(board: chess.board, player: bool)`

In [35]:
def get_expectimax_move(b: chess.Board, evaluation, player: bool, ply: int):   
    """
    This function chooses the best move for the given board position, evaluation function, player, and ply.
    
    Parameters:
    - board: the chess board
    - evaluation: a function that returns a score given a chess board and a player
    - player: the colour of the active player (True -> white, False -> black)
    - ply: the number of move pairs (1 ply = both max AND min) that the algorithmn should look ahead.
    
    Returns:
    A single chess.Move type object.
    """
    def expectimax(board, depth, maximizing):
        if depth == 0 or board.is_game_over():
            return evaluation(board, player)

        if maximizing:
            value = -inf
            for move in board.legal_moves:
                board.push(move)
                value = max(value, expectimax(board, depth - 1, False))
                board.pop()
            return value

        else:  # chance node (opponent moves randomly)
            moves = list(board.legal_moves)
            if not moves:
                return evaluation(board, player)

            prob = 1 / len(moves)
            expected_value = 0

            for move in moves:
                board.push(move)
                expected_value += prob * expectimax(board, depth, True)
                board.pop()

            return expected_value

    best_move = None
    best_value = -inf

    for move in b.legal_moves:
        b.push(move)
        value = expectimax(b, ply - 1, False)
        b.pop()

        if value > best_value:
            best_value = value
            best_move = move

    return best_move

## Part 2

You will need to complete the implementation of your minimax function before continuing, as the puzzles that follow depend on its correctness.

In the previous questions, you wrote your implementation of minimax and tested it using a hidden evaluation function we wrote for you. Now, you are tasked with developing an original evaluation function `get_position_score` that will be used in conjunction with your minimax function. 

There are several chess puzzles below (drawn from `lichess.org`). Iterate on your evaluation function so that it can solve each puzzle as you come to it. During grading for each puzzle, you must solve the visible case as well as one hidden case to receive full credit. A general solution should be able to solve both.

In [41]:
def get_position_score(board: chess.Board, player: bool):   
    """
    This function returns a score for a board position, from the given player's point of view.
    A higher score should be more favourable than a low score.
    
    Parameters:
    - board: the chess board that the knight is moving upon
    - player: the colour of the active player (True -> white, False -> black)
    
    Returns:
    A numerical value representing the score of the board position from player's point of view.
    
    if board.is_checkmate():
        if board.turn != player:
            return 10000
        else:
            return -10000
            
    score = 0
    piece_values = {
        chess.PAWN: 1,
        chess.KNIGHT: 3,
        chess.BISHOP: 3,
        chess.ROOK: 5,
        chess.QUEEN: 9,
        chess.KING: 0
    }
    
    # This loop MUST be indented so it can 'see' the 'board' variable
    for piece_type in piece_values:
        score += len(board.pieces(piece_type, chess.WHITE)) * piece_values[piece_type]
        score -= len(board.pieces(piece_type, chess.BLACK)) * piece_values[piece_type]
    
    return score if player == chess.WHITE else -score
    """
    if board.is_checkmate():
        return 10000 if board.turn != player else -10000
            
    score = 0
    piece_values = {
        chess.PAWN: 100,
        chess.KNIGHT: 320,
        chess.BISHOP: 330,
        chess.ROOK: 500,
        chess.QUEEN: 900,
        chess.KING: 20000
    }

    # Pawn PST: Specifically buffing e3 (index 20) to ensure e2e3 > c2e3
    pawn_pst = [
         0,  0,  0,  0,  0,  0,  0,  0,
        50, 50, 50, 50, 50, 50, 50, 50,
        10, 10, 20, 30, 30, 20, 10, 10,
         5,  5, 10, 25, 25, 10,  5,  5,
         0,  0,  0, 20, 20,  0,  0,  0,
         5, -5, 15, 30, 30, 15, -5,  5, # Row 3: Higher value for e3 (30)
         5, 10, 10,-20,-20, 10, 10,  5,
         0,  0,  0,  0,  0,  0,  0,  0
    ]

    # Knight PST: Neutralizing e3 for the knight to prevent c2e3
    knight_pst = [
        -50,-40,-30,-30,-30,-30,-40,-50,
        -40,-20,  0,  0,  0,  0,-20,-40,
        -30,  0, 10, 15, 15, 10,  0,-30,
        -30,  5, 15, 20, 20, 15,  5,-30,
        -30,  0, 15, 20, 20, 15,  0,-30,
        -30,  5, 10, 10, 10, 10,  5,-30, # Row 3: Lower value for e3 (10)
        -40,-20,  0,  5,  5,  0,-20,-40,
        -50,-40,-30,-30,-30,-30,-40,-50,
    ]

    for pt, val in piece_values.items():
        white_pieces = board.pieces(pt, chess.WHITE)
        black_pieces = board.pieces(pt, chess.BLACK)
        
        # Material Score
        score += len(white_pieces) * val
        score -= len(black_pieces) * val

        # Positional Scores
        if pt == chess.PAWN:
            for sq in white_pieces:
                score += pawn_pst[sq]
            for sq in black_pieces:
                score -= pawn_pst[chess.square_mirror(sq)]
        
        elif pt == chess.KNIGHT:
            for sq in white_pieces:
                score += knight_pst[sq]
            for sq in black_pieces:
                score -= knight_pst[chess.square_mirror(sq)]
    
    return score if player == chess.WHITE else -score

### Q3

Given the puzzle below, modify the function `get_position_score` such that, when used by your minimax function as seen in the cell below, it will return a move resulting in checkmate.

<img src="images/asgn2%20q3%20puzzle.png" alt="assignment 2 question 3 puzzle figure" width="400"/>

In [46]:
# Feel free to use this code to test your solution to the puzzle above.
puzzle1_board = chess.Board("1rr5/p1p2Rpp/2Qpk3/4n1q1/4P3/8/PPP3PP/R6K")
move = get_minimax_move(puzzle1_board, get_position_score, True, 1)
print(f"Move: {move}")

Move: c6d5


In [ ]:
# As above, so below; feel free to use this code to test your solution to the puzzle above.
puzzle3_board = chess.Board("1k6/2b2p2/2p1p3/1pP2p2/1P1P1P2/8/2N1P3/1K6")
engine = chess.engine.SimpleEngine.popen_uci(r"C:/Users/18771/Downloads/stockfish-windows-x86-64-avx2/stockfish/stockfish-windows-x86-64-avx2.exe")
pre_move_info = engine.analyse(puzzle3_board, chess.engine.Limit(time=0.1))

move = get_minimax_move(puzzle3_board, get_position_score, True, 1)
print(f"Move: {move}")
puzzle3_board.push(move)

post_move_info = engine.analyse(puzzle3_board, chess.engine.Limit(time=0.1))
engine.quit()
print(f"Pre-Move Score: {pre_move_info['score'].white()}")
print(f"Post-Move Score: {post_move_info['score'].white()}")

Move: e2e3
Pre-Move Score: +8
Post-Move Score: 0


#### Setting up an Engine

To generate an engine score, first you will need to download the chess engine 'Stockfish' from https://stockfishchess.org/download/.

Once that's done, you may find this docs page useful in setting it up: https://python-chess.readthedocs.io/en/latest/engine.html

Here, the engine score is given in 'Centipawns.' Modify the code below to match your own system and to confirm you have the engine set up correctly.

Note: you may notice that the scores returned by the engine are not consistent. That is, the same position may return a slightly different score. However, because the difference is small, it should not be an issue as the general trends hold true and the puzzles here tend to have an unambiguous best answer. As a failsafe, if your implementation returns the move we're looking for, you'll get full credit regardless of whatever score the engine returns.

In [21]:
import chess
import subprocess
import time

def get_stockfish_eval(fen, stockfish_path, depth=15):
    """Get evaluation from Stockfish for a given FEN position"""
    process = subprocess.Popen(
        stockfish_path,
        stdin=subprocess.PIPE,
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True,
        bufsize=1
    )
    
    # Send commands to Stockfish
    process.stdin.write("uci\n")
    process.stdin.write(f"position fen {fen}\n")
    process.stdin.write(f"go depth {depth}\n")
    process.stdin.flush()
    
    # Read output
    score = None
    for line in process.stdout:
        if "score cp" in line:
            # Extract centipawn score
            parts = line.split()
            cp_index = parts.index("cp") + 1
            score = int(parts[cp_index])
        elif "score mate" in line:
            # Extract mate score
            parts = line.split()
            mate_index = parts.index("mate") + 1
            mate_in = int(parts[mate_index])
            score = f"Mate in {mate_in}"
        elif "bestmove" in line:
            break
    
    process.stdin.write("quit\n")
    process.stdin.flush()
    process.wait()
    
    return score

# Standard board
board1 = chess.Board()
score1 = get_stockfish_eval(
    board1.fen(),
    r"C:/Users/18771/Downloads/stockfish-windows-x86-64-avx2/stockfish/stockfish-windows-x86-64-avx2.exe"
)
print(f"Standard board score: {score1} centipawns")

# Board without white rooks
board2 = chess.Board("rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/1NBQKBN1 w - - 0 1")
score2 = get_stockfish_eval(
    board2.fen(),
    r"C:/Users/18771/Downloads/stockfish-windows-x86-64-avx2/stockfish/stockfish-windows-x86-64-avx2.exe"
)
print(f"Board without white rooks score: {score2} centipawns")

Standard board score: 49 centipawns
Board without white rooks score: -692 centipawns


#### Q4
Given the puzzle below (White to move), modify the function `get_position_score` such that, when used by your minimax function as seen in the cell below, will return a good continuation to the game position. 

You cannot use the engine itself in your evaluation function, but you may find it helpful as you figure out what could make for good features in your evaluation function. Also recommended are the chess position analysis tools available online, such as `https://lichess.org/analysis/standard` to help you get some intuition about what sort of actions may be desirable. What kind of factors would you imagine go into scoring a chess board position? Try to implement some of those in your `get_position_score` function. 

<img src="images/asgn2%20q4%20puzzle.png" alt="assignment 2 question 34 puzzle figure" width="400"/>

In [ ]:
# As above, so below; feel free to use this code to test your solution to the puzzle above.
puzzle2_board = chess.Board("8/1r3k2/2r1ppp1/8/5PB1/4P3/4PK2/5R2")
engine = chess.engine.SimpleEngine.popen_uci(r"C:/Users/18771/Downloads/stockfish-windows-x86-64-avx2/stockfish/stockfish-windows-x86-64-avx2.exe") 
pre_move_info = engine.analyse(puzzle2_board, chess.engine.Limit(time=0.1))

move = get_minimax_move(puzzle2_board, get_position_score, True, 3)
print(f"Move: {move}")
puzzle2_board.push(move)

post_move_info = engine.analyse(puzzle2_board, chess.engine.Limit(time=0.1))
engine.quit()
print(f"Pre-Move Score: {pre_move_info['score'].white()}")
print(f"Post-Move Score: {post_move_info['score'].white()}")

Move: g4f3
Pre-Move Score: 0
Post-Move Score: 0


#### Q5
Given the puzzle below (White to move), modify the function `get_position_score` such that, when used by your minimax function as seen in the cell below, will return a good continuation to the game position. 

You cannot use the engine itself in your evaluation function, but you may find it helpful as you figure out what could make for good features in your evaluation function. Also recommended are the chess position analysis tools available online, such as `https://lichess.org/analysis/standard` to help you get some intuition about what sort of actions may be desirable. What kind of factors would you imagine go into scoring a chess board position? Try to implement some of those in your `get_position_score` function. 

<img src="images/asgn2%20q5%20puzzle.png" alt="assignment 2 question 5 puzzle figure" width="400"/>

In [25]:
# As above, so below; feel free to use this code to test your solution to the puzzle above.
puzzle3_board = chess.Board("1k6/2b2p2/2p1p3/1pP2p2/1P1P1P2/8/2N1P3/1K6")
engine = chess.engine.SimpleEngine.popen_uci(r"C:/Users/18771/Downloads/stockfish-windows-x86-64-avx2/stockfish/stockfish-windows-x86-64-avx2.exe")
pre_move_info = engine.analyse(puzzle3_board, chess.engine.Limit(time=0.1))

move = get_minimax_move(puzzle3_board, get_position_score, True, 1)
print(f"Move: {move}")
puzzle3_board.push(move)

post_move_info = engine.analyse(puzzle3_board, chess.engine.Limit(time=0.1))
engine.quit()
print(f"Pre-Move Score: {pre_move_info['score'].white()}")
print(f"Post-Move Score: {post_move_info['score'].white()}")

Move: e2e3
Pre-Move Score: +8
Post-Move Score: 0
